In [1]:
import numpy as np
import fastobo
import wget

In [6]:
wget.download("http://purl.obolibrary.org/obo/go/go.obo", '../data/GO/go.obo')

'../data/GO/go.obo'

In [7]:
obodoc = fastobo.load('../data/GO/go.obo')

SyntaxError: expected QuotedString (go.obo, line 274469)

In [2]:
import networkx
import obonet

In [3]:
url = "http://purl.obolibrary.org/obo/go/go-basic.obo"
graph = obonet.read_obo(url)

In [4]:
goid_to_category = {id_: data.get('namespace') for id_, data in graph.nodes(data=True)}

In [5]:
%%time
goid_to_category['GO:0000331']

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 17.9 µs


'cellular_component'

In [6]:
import pandas as pd

In [16]:
df = pd.read_csv('../data/GO/pdb_chain_go.csv', skiprows = 1, error_bad_lines=False, warn_bad_lines = False)

In [17]:
df.head()

,PDB,CHAIN,SP_PRIMARY,WITH_STRING,EVIDENCE,GO_ID
0,101m,A,IPRO,InterPro:IPR000971,IEA,GO:0020037
1,101m,A,IPRO,InterPro:IPR002335,IEA,GO:0015671
2,101m,A,IPRO,InterPro:IPR002335,IEA,GO:0019825
3,101m,A,IPRO,InterPro:IPR002335,IEA,GO:0020037
4,101m,A,IPRO,InterPro:IPR012292,IEA,GO:0019825


In [20]:
%%time
df = df[['PDB', 'GO_ID']]
pdb_to_go = {}
for key, value in df.values[:100]:
    go_list = pdb_to_go.get(key, set())
    go_list.add(value)
    pdb_to_go[key] = go_list
    
pdb_to_go

CPU times: user 382 ms, sys: 330 ms, total: 712 ms
Wall time: 710 ms


{'101m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '102l': {'GO:0003796',
  'GO:0009253',
  'GO:0016998',
  'GO:0019076',
  'GO:0019835'},
 '102m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '103l': {'GO:0003796',
  'GO:0009253',
  'GO:0016998',
  'GO:0019076',
  'GO:0019835'},
 '103m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '104l': {'GO:0003796',
  'GO:0009253',
  'GO:0016998',
  'GO:0019076',
  'GO:0019835'},
 '104m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '105m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '106m': {'GO:0005344',
  'GO:0015671',
  'GO:0019825',
  'GO:0020037',
  'GO:0046872'},
 '107l': {'GO:0003796',
  'GO:0009253',
  'GO:0016998',
  'GO:0019076',
  'GO:0019835'}}

In [103]:
%%time
ids = np.unique(pdb_to_go[pdb_to_go[:, 0] == '101m', 1])
ids

CPU times: user 531 ms, sys: 11.8 ms, total: 543 ms
Wall time: 539 ms


array(['GO:0005344', 'GO:0015671', 'GO:0019825', 'GO:0020037',
       'GO:0046872'], dtype=object)

In [104]:
pdb_go_dict = {}
for pdb, go in pdb_to_go[0]:
    pdb_go_dict[pdb] = pdb_go_dict.get(pdb, set()).add(go)
pdb_go_dict

ValueError: too many values to unpack (expected 2)

In [53]:

for go_id in ids:
    print(go_id, goid_to_category[go_id])

GO:0020037 molecular_function
GO:0015671 biological_process
GO:0019825 molecular_function
GO:0020037 molecular_function
GO:0019825 molecular_function
GO:0020037 molecular_function
GO:0046872 molecular_function
GO:0005344 molecular_function
GO:0015671 biological_process


In [56]:
import os
import scipy.sparse

In [79]:
%%time
X_mf = []
y_mf = []
X_cc = []
y_cc = []
X_bp = []
y_bp = []
for file in os.listdir('../data/contact_maps/sparse_matrices')[:2]:
    if file.endswith('.npz'):
        pdb_code = file.split('.')[0]
        go_ids = pdb_to_go[pdb_to_go['PDB'] == pdb_code.lower()]['GO_ID'].unique()
        X_temp = scipy.sparse.load_npz('../data/contact_maps/sparse_matrices/' + file).todense()
        y_mf_temp = []
        y_cc_temp = []
        y_bp_temp = []
        for go_id in go_ids:
            category = goid_to_category[go_id]
            if category == 'molecular_function':
                y_mf_temp.append(go_id)
            if category == 'cellular_component':
                y_cc_temp.append(go_id)
            if category == 'biological_process':
                y_bp_temp.append(go_id)
        if len(y_mf_temp) > 0:
            X_mf.append(X_temp)
            y_mf.append(y_mf_temp)
        if len(y_cc_temp) > 0:
            X_cc.append(X_temp)
            y_cc.append(y_cc_temp)
        if len(y_bp_temp) > 0:
            X_bp.append(X_temp)
            y_bp.append(y_bp_temp)

print(X_mf, y_mf)
print(X_cc, y_cc)
print(X_bp, y_bp)

[matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 1, 0, 1],
        [0, 0, 0, ..., 1, 1, 0]]), matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])] [['GO:0003676', 'GO:0004523', 'GO:0003964'], ['GO:0003677', 'GO:0003707', 'GO:0004879']]
[matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])] [['GO:0005634']]
[matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 1, 0, 1],
        [0, 0, 0, ..., 1, 1, 0]]), matrix([[0, 1, 1, ..., 0, 0, 0],
      